In [33]:
# %pip install --upgrade openai

In [17]:
from dotenv import load_dotenv
load_dotenv()

import os
from openai import OpenAI
from gpt import (
    create_file_search_assistant,
    upload_file_to_vectore_store,
    assistant_use_vector_store,
    create_thread,
)

In [18]:
MODEL = "gpt-4o"
FILE = "thales_t1.pdf"
client = OpenAI(
    api_key=os.environ.get("API_KEY")
)

In [19]:
assistant = create_file_search_assistant(
    client=client,
    model=MODEL,
    assistant_name="Financial Analyst Assistant",
    instructions="You are an expert financial & sales analyst. Use you knowledge base to extract and summarize information about audited financial statements",
)

create_file_search_assistant took 0.6110286712646484 seconds


In [20]:
vector_store = upload_file_to_vectore_store(
    client=client,
    vector_store_name="Financial Statements",
    file_paths=[FILE],
)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
upload_file_to_vectore_store took 3.3434910774230957 seconds


In [21]:
assistant = assistant_use_vector_store(
    client=client,
    assistant=assistant,
    vector_store=vector_store,
)

assistant_use_vector_store took 0.3343784809112549 seconds


In [22]:
thread = create_thread(
    client=client,
    message_content="What are the revenues by sector on the first quarter of 2024 and how does it compare to the first quarter of 2023",
)

None
create_thread took 0.30292797088623047 seconds


In [23]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("="*50)
print("\n".join(citations))

### Revenue Comparison by Sector: Q1 2024 vs Q1 2023

The revenue of Thales for Q1 2024 is classified into several sectors, and the comparison with Q1 2023 is as follows:

#### Aerospace
- **Q1 2024:** €1,182 million
- **Q1 2023:** €1,156 million
- **Change:** +2.3% (Published), +5.1% (Organic)

#### Defense & Security
- **Q1 2024:** €2,305 million
- **Q1 2023:** €2,037 million
- **Change:** +13.1% (Published), +13.4% (Organic)

#### Digital Identity & Security
- **Q1 2024:** €916 million
- **Q1 2023:** €818 million
- **Change:** +12.0% (Published), -2.5% (Organic)**

#### Others
- **Q1 2024:** €18 million
- **Q1 2023:** €15 million
- **Change:** Not Specified (NS)

#### Total Revenue
- **Q1 2024:** €4,421 million
- **Q1 2023:** €4,026 million
- **Change:** +9.8% (Published), +7.9% (Organic)

### Summary

- **Aerospace** experienced moderate growth mainly driven by the aviation sector.
- **Defense & Security** saw significant growth largely due to increased orders and favorable market 